In [1]:
from pyiron_base import Project, GenericJob, GenericParameters
import numpy as np

In [2]:
import damask
from damask import Geom
from damask import seeds

In [3]:
import h5py

In [4]:
import yaml

In [5]:
with open('Examples/material.yaml') as f:
    material = yaml.load(f, Loader=yaml.FullLoader)
with open('Examples/tensionX.load') as f:
    tension = f.readlines()

In [6]:
import os

In [7]:
class DAMASKjob(GenericJob):
    def __init__(self, project, job_name):
        super(DAMASKjob, self).__init__(project, job_name)
        self.input = GenericParameters(table_name="input")
        self.input['C11'] = 0
        self.input['C12'] = 0
        self.input['C44'] = 0
        self.input['grid'] = np.array([16,16,16])
        self.input['size'] = np.array([1.0,1.0,1.0])
        self.input['grains'] = 20
        self.executable = "DAMASK_grid -l tensionX.load -g damask.geom"
        self._material = None 
        self._tension = None 
        
    @property
    def material(self):
        return self._material
    
    @material.setter
    def material(self, mat):
        self._material = mat
        
    @property
    def tension(self):
        return self._tension
    
    @tension.setter
    def tension(self, tension):
        self._tension = tension
        
    def write_input(self): 
        with open(os.path.join(self.working_directory, 'material.yaml'), "w") as f:
            yaml.dump(self._material, f)
        with open(os.path.join(self.working_directory, 'tensionX.load'), "w") as f:
            f.writelines(self._tension)
        seed = seeds.from_random(self.input['size'], self.input['grains'])
        new_geom = Geom.from_Voronoi_tessellation(self.input['grid'], self.input['size'], seed)
        new_geom.save_ASCII(os.path.join(self.working_directory, "damask.geom"))
        new_geom.save(os.path.join(self.working_directory, "damask"))
        C_matrix = [self.input['C11']*1e9, self.input['C12']*1e9, self.input['C44']*1e9]
        elasticity={}
        elasticity.update({'type': 'hooke'})
        elastic_constants = {'C_11': C_matrix[0], 'C_12': C_matrix[1], 'C_44': C_matrix[2]}
        elasticity.update(elastic_constants)
        mat = damask.Config.load(os.path.join(self.working_directory, 'material.yaml'))
        mat['phase']['Aluminum']['elasticity'] = elasticity
        mat.save(os.path.join(self.working_directory, 'material.yaml'))
        
    def collect_output(self): 
        pass 
    
    def to_hdf(self, hdf=None, group_name=None): 
        super().to_hdf(
            hdf=hdf,
            group_name=group_name
        )
        with self.project_hdf5.open("input") as h5in:
            self.input.to_hdf(h5in)
            h5in["material"] = self._material
            h5in["tension"] = self._tension

    def from_hdf(self, hdf=None, group_name=None): 
        super().from_hdf(
            hdf=hdf,
            group_name=group_name
        )
        with self.project_hdf5.open("input") as h5in:
            self.input.from_hdf(h5in)
            self._material = h5in["material"]
            self._tension = h5in["tension"]
    
    @property
    def output_file(self):
        file_name = os.path.join(self.working_directory, "damask_tensionX.hdf5")
        if self.status.finished and os.path.exists(file_name):
            return file_name
    
    @property
    def output(self):
        file_name = self.output_file
        if file_name is not None:
            return damask.Result(file_name)

    def eval_stress(self):
        """
        return the stress as a numpy array
        Parameters
        ----------
        job_file : str
          Name of the job_file
        """
        file_name = self.output_file
        if file_name is not None:
            d = damask.Result(file_name)
            stress_path = d.get_dataset_location('avg_sigma')
            stress = np.zeros(len(stress_path))
            hdf = h5py.File(d.fname)
            for count,path in enumerate(stress_path):
                stress[count] = np.array(hdf[path])
            stress = np.array(stress)/1E6
            return stress

    def eval_strain(self):
        """
        return the strain as a numpy array
        Parameters
        ----------
        job_file : str
          Name of the job_file
        """
        file_name = self.output_file
        if file_name is not None:
            d = damask.Result(file_name)
            stress_path = d.get_dataset_location('avg_sigma')
            strain = np.zeros(len(stress_path))
            hdf = h5py.File(d.fname)
            for count,path in enumerate(stress_path):
                strain[count] = np.array(hdf[path.split('avg_sigma')[0]     + 'avg_epsilon'])

            return strain  

In [8]:
pr = Project("test")

In [9]:
# pr.remove_jobs_silently(recursive=True)

In [10]:
job = pr.create_job(DAMASKjob, "damask")
job.material = material
job.tension = tension
job.input['C11'] = 110.193270
job.input['C12'] = 61.752630
job.input['C44'] = 32.746691
job.run()

/srv/conda/envs/notebook/lib/python3.7/site-packages/pyiron_base/generic/parameters.py:860: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self._dataset[col] = np.array(self._dataset[col]).tolist()
2020-11-04 11:42:34,945 - pyiron_log - WARNING - The job damask is being loaded instead of running. To re-run use the argument 'delete_existing_job=True in create_job'


In [11]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,18,finished,None,damask,/damask,/home/jovyan/,test/,2020-11-04 11:28:43.220686,2020-11-04 11:33:55.335456,312.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2ddamask-2dceiy0z9t#1,GenericJob,0.4,None,None


In [12]:
damask_results = job.output

In [13]:
damask_results.add_Cauchy()
damask_results.add_strain_tensor()
damask_results.add_Mises('sigma')
damask_results.add_Mises('epsilon_V^0.0(F)')
damask_results.add_calculation('avg_sigma',"np.average(#sigma_vM#)")
damask_results.add_calculation('avg_epsilon',"np.average(#epsilon_V^0.0(F)_vM#)")
damask_results.save_vtk(['sigma','epsilon_V^0.0(F)','sigma_vM','epsilon_V^0.0(F)_vM'])

 █░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   2% ETA 0:00:16

Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).


 ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   6% ETA 0:00:11

Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).


 ████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   9% ETA 0:00:09

...

 ████████████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  32% ETA 0:00:01

Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (na

 ██████████████████████████████████████████████████ 100% ETA 0:00:00

Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (name already exists).
Could not add dataset: Unable to create link (na


 ██████████████████████████████████████████████████ 100% ETA 0:00:00


In [14]:
job.eval_stress()

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:97: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


array([9.00410083e-12, 1.92708289e+01, 3.85620251e+01, 5.73577154e+01,
       6.84940533e+01, 7.30485177e+01, 7.55024618e+01, 7.72823816e+01,
       7.87604622e+01, 8.00280194e+01, 8.11269630e+01, 8.20720459e+01,
       8.28833720e+01, 8.35781218e+01, 8.41765652e+01, 8.46934781e+01,
       8.51447138e+01, 8.55418707e+01, 8.58943643e+01, 8.62099729e+01,
       8.64935503e+01, 8.67506011e+01, 8.69841622e+01, 8.71976855e+01,
       8.73933917e+01, 8.75737825e+01, 8.77407662e+01, 8.78958898e+01,
       8.80406147e+01, 8.81761327e+01, 8.83020858e+01, 8.84241085e+01,
       8.85349555e+01, 8.86432696e+01, 8.87467958e+01, 8.88417595e+01,
       8.89355132e+01, 8.90219998e+01, 8.91097216e+01, 8.91894900e+01,
       8.92682041e+01, 8.93416891e+01, 8.94154582e+01, 8.94855219e+01,
       8.95567227e+01, 8.96221758e+01, 8.96865188e+01, 8.97475241e+01,
       8.98090793e+01, 8.98683906e+01, 8.99272648e+01, 8.99839222e+01,
       9.00400803e+01, 9.00945998e+01, 9.01486411e+01, 9.02013205e+01,
      

In [15]:
job.eval_strain()

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:116: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


array([0.        , 0.00022344, 0.00044684, 0.00067121, 0.00091067,
       0.00116786, 0.00143406, 0.00170477, 0.00197811, 0.00225313,
       0.00252936, 0.00280644, 0.00308417, 0.00336238, 0.00364098,
       0.00391985, 0.00419895, 0.00447822, 0.00475761, 0.00503711,
       0.00531666, 0.00559626, 0.00587589, 0.00615552, 0.00643515,
       0.00671478, 0.00699438, 0.00727396, 0.00755352, 0.00783305,
       0.00811254, 0.00839201, 0.00867141, 0.00895077, 0.00923011,
       0.0095094 , 0.00978863, 0.01006782, 0.01034694, 0.01062602,
       0.01090504, 0.01118402, 0.01146294, 0.01174184, 0.01202061,
       0.01229936, 0.01257804, 0.01285667, 0.01313526, 0.01341379,
       0.01369225, 0.01397065, 0.01424898, 0.01452726, 0.01480547,
       0.01508361, 0.01536169, 0.01563971, 0.01591766, 0.01619555,
       0.01647337, 0.01675112, 0.01702881, 0.01730643, 0.01758398,
       0.01786147, 0.01813889, 0.01841624, 0.01869353, 0.01897074,
       0.01924789, 0.01952497, 0.01980198, 0.02007892, 0.02035